# Instalación de Paquetes

Este notebook requiere los siguientes paquetes: **streamlit**, **mysql-connector-python**, **Faker**, **wordcloud** y **plotly**. Ejecuta la siguiente celda para instalarlos.

In [3]:
!pip install streamlit mysql-connector-python Faker wordcloud plotly mysql

Defaulting to user installation because normal site-packages is not writeable
  Using cached mysql-0.0.3-py3-none-any.whl.metadata (746 bytes)
Using cached mysql-0.0.3-py3-none-any.whl (1.2 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


# Creación de la Base de Datos y Tablas
asasas
En esta celda se conecta a MySQL (usando los parámetros indicados) y se crea la base de datos `health_db` con las siguientes tablas:
- **states:** Estados de la república mexicana
- **pathologies:** Diversas patologías
- **patients:** Información de pacientes (incluye un campo `palabras` para la nube de palabras)
- **patient_pathologies:** Tabla relacional que asocia pacientes con sus patologías

In [3]:
import mysql.connector
from faker import Faker
import random

# Conexión a MySQL (sin seleccionar base, para poder crearla)
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="PruebaEstu2012"
)
cursor = conn.cursor()

# Crear base de datos y usarla
cursor.execute("CREATE DATABASE IF NOT EXISTS health_db")
cursor.execute("USE health_db")

# Crear tabla de estados (con coordenadas para el mapa)
cursor.execute("""
CREATE TABLE IF NOT EXISTS states (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6)
)
""")

# Crear tabla de patologías
cursor.execute("""
CREATE TABLE IF NOT EXISTS pathologies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL
)
""")

# Crear tabla de pacientes
cursor.execute("""
CREATE TABLE IF NOT EXISTS patients (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    age INT,
    sex ENUM('M', 'F') NOT NULL,
    state_id INT,
    palabras TEXT,
    registration_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (state_id) REFERENCES states(id)
)
""")

# Crear tabla relacional entre pacientes y patologías
cursor.execute("""
CREATE TABLE IF NOT EXISTS patient_pathologies (
    patient_id INT,
    pathology_id INT,
    PRIMARY KEY (patient_id, pathology_id),
    FOREIGN KEY (patient_id) REFERENCES patients(id),
    FOREIGN KEY (pathology_id) REFERENCES pathologies(id)
)
""")

# Diccionario de estados con coordenadas aproximadas (latitud, longitud)
estados = {
    "Aguascalientes": (21.8853, -102.2916),
    "Baja California": (32.6243, -115.4523),
    "Baja California Sur": (26.0448, -111.6660),
    "Campeche": (19.8450, -90.5238),
    "Chiapas": (16.7569, -93.1290),
    "Chihuahua": (28.6353, -106.0889),
    "Ciudad de México": (19.4326, -99.1332),
    "Coahuila": (27.0659, -101.6920),
    "Colima": (19.2433, -103.7247),
    "Durango": (24.0273, -104.6532),
    "Estado de México": (19.3325, -99.6333),
    "Guanajuato": (21.0190, -101.2570),
    "Guerrero": (17.4210, -99.6300),
    "Hidalgo": (20.0914, -98.7621),
    "Jalisco": (20.6597, -103.3496),
    "Michoacán": (19.5665, -101.7050),
    "Morelos": (18.9261, -99.2302),
    "Nayarit": (21.7510, -105.2324),
    "Nuevo León": (25.5920, -99.9961),
    "Oaxaca": (17.0732, -96.7266),
    "Puebla": (19.0413, -98.2062),
    "Querétaro": (20.5888, -100.3899),
    "Quintana Roo": (19.1810, -88.4791),
    "San Luis Potosí": (22.1565, -100.9855),
    "Sinaloa": (24.8091, -107.3940),
    "Sonora": (29.0745, -110.9559),
    "Tabasco": (18.1453, -92.4614),
    "Tamaulipas": (25.3548, -99.0731),
    "Tlaxcala": (19.3133, -98.2400),
    "Veracruz": (19.1738, -96.1342),
    "Yucatán": (21.1619, -89.6675),
    "Zacatecas": (22.7709, -102.5833)
}

# Insertar estados
for state_name, (lat, lon) in estados.items():
    cursor.execute("INSERT INTO states (name, latitude, longitude) VALUES (%s, %s, %s)", (state_name, lat, lon))

# Insertar patologías (se mantienen las 6 definidas)
pathologies = ["Cáncer", "Diabetes", "Obesidad", "Hipertensión", "Asma", "Enfermedad Cardiovascular"]
for pathology in pathologies:
    cursor.execute("INSERT INTO pathologies (name) VALUES (%s)", (pathology,))

conn.commit()

# ---------------------------------------------------------------------------
# Lista de palabras para la nube (50 en total: 15 importantes y 35 adicionales)
# ---------------------------------------------------------------------------
important_words = [
    "dolor", "trauma", "esperanza", "recuperación", "confianza",
    "ánimo", "resiliencia", "optimismo", "apoyo", "cuidado",
    "emoción", "soporte", "fortaleza", "alegría", "alivio"
]

other_words = [
    "desesperanza", "miedo", "incertidumbre", "soledad", "estrés",
    "ansiedad", "inseguridad", "temor", "inquietud", "compasión",
    "empatía", "satisfacción", "contento", "amor", "cariño",
    "confusión", "frustración", "desilusión", "decepción", "vitalidad",
    "energía", "paciencia", "cambio", "nerviosismo", "pasión",
    "serenidad", "paz", "lucha", "victoria", "dificultad",
    "determinación", "motivación", "inspiración", "entusiasmo", "curación"
]

# Combinamos ambas listas y asignamos pesos: 3 para cada palabra importante y 1 para las demás.
all_words = important_words + other_words
weights = [3] * len(important_words) + [1] * len(other_words)

# ---------------------------------------------------------------------------
# Generar 1000 observaciones de pacientes con mayor variedad
# ---------------------------------------------------------------------------
fake = Faker('es_ES')

# Obtener IDs de estados y patologías
cursor.execute("SELECT id FROM states")
state_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT id FROM pathologies")
pathology_ids = [row[0] for row in cursor.fetchall()]

for _ in range(1000):
    name = fake.name()
    # Usar una distribución gaussiana para la edad
    age = int(random.gauss(45, 15))
    # Asegurar que la edad esté entre 18 y 90
    age = max(18, min(age, 90))
    sex = random.choice(['M', 'F'])
    state_id = random.choice(state_ids)
    
    # Generar "palabras" compuestas de 1 a 3 oraciones.
    n_sentences = random.randint(1, 3)
    sentences = []
    for _ in range(n_sentences):
        n_words = random.randint(8, 20)
        sentence = " ".join(random.choices(all_words, weights=weights, k=n_words))
        # Capitalizar la oración y agregar un punto final.
        sentence = sentence.capitalize() + "."
        sentences.append(sentence)
    palabras = " ".join(sentences)
    
    # Generar una fecha de registro aleatoria en el último año
    registration_date = fake.date_time_between(start_date="-1y", end_date="now")
    
    cursor.execute("""
        INSERT INTO patients (name, age, sex, state_id, palabras, registration_date)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (name, age, sex, state_id, palabras, registration_date))
    patient_id = cursor.lastrowid
    
    # Asignar de 0 a 3 patologías aleatorias al paciente (permitiendo que algunos pacientes no tengan ninguna)
    num_pathologies = random.randint(0, 3)
    if num_pathologies > 0:
        assigned_pathologies = random.sample(pathology_ids, num_pathologies)
        for pid in assigned_pathologies:
            cursor.execute("INSERT INTO patient_pathologies (patient_id, pathology_id) VALUES (%s, %s)", (patient_id, pid))

conn.commit()
cursor.close()
conn.close()

print("Base de datos 'health_db' creada y poblada con 1000 observaciones con mayor variedad.")


Base de datos 'health_db' creada y poblada con 1000 observaciones con mayor variedad.


streamlit run appstream.py 